In [0]:
def precision_recall(gt,preds):
    TP=0
    FP=0
    FN=0
    for t in gt:
        if t in preds:
            TP+=1
        else:
            FN+=1
    for p in preds:
        if p not in gt:
            FP+=1
    if TP+FP==0:
        precision=0
    else:
        precision=TP/float(TP+FP)
    if TP+FN==0:
        recall=0
    else:
        recall=TP/float(TP+FN)
    return precision,recall

In [0]:
Y

['[28, 12]',
 '[16, 35, 878]',
 '[99, 878]',
 '[80, 12]',
 '[35, 12]',
 '[28, 12, 53]',
 '[18, 35]',
 '[16]',
 '[10402, 80]',
 '[99]',
 '[27, 53, 35]',
 '[28, 80]',
 '[16, 35, 10751]',
 '[28, 12, 37]',
 '[16]',
 '[16]',
 '[80, 18]',
 '[18]',
 '[16]',
 '[80, 35]',
 '[28, 35, 80]',
 '[99, 12]',
 '[12]',
 '[28, 35]',
 '[18, 9648, 53]',
 '[28, 18, 36, 10752]',
 '[28, 12, 35, 878]',
 '[35, 10751, 16]',
 '[28, 27, 878]',
 '[16]',
 '[18, 28, 80, 10749]',
 '[35]',
 '[10751, 16, 14]',
 '[99]',
 '[99]',
 '[12, 16, 99]',
 '[16]',
 '[16, 18, 14, 27]',
 '[28, 37]',
 '[99]',
 '[18, 35, 10751]',
 '[35, 18]',
 '[28, 80, 53]',
 '[28, 27]',
 '[99]',
 '[28, 35, 9648, 53]',
 '[35, 10749]',
 '[28, 35, 80]',
 '[28, 878]',
 '[28, 12, 80]',
 '[35, 10751]',
 '[18, 99, 10770]',
 '[99, 10751]',
 '[28, 53]',
 '[14, 27, 28, 53]',
 '[10751, 12]',
 '[28, 12, 18, 10751, 10770]',
 '[99]',
 '[53, 27, 18]',
 '[18, 36]',
 '[80, 18, 53]',
 '[28, 27, 878, 53]',
 '[35, 10749]',
 '[99]',
 '[12, 35, 10749]',
 '[28, 37]',
 '[1

In [0]:
mask = np.random.rand(len(X)) < 0.8


In [2]:
from sklearn.preprocessing import MultiLabelBinarizer
mlb=MultiLabelBinarizer()
Y=mlb.fit_transform(Y)

Y.shape

NameError: ignored

In [0]:
X_train=X[mask]
X_test=X[~mask]
Y_train=Y[mask]
Y_test=Y[~mask]

In [0]:
del df
del mlb
del Y
del pickle
del np 
del pd
del X
del MultiLabelBinarizer

In [0]:
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras import optimizers
model_visual = Sequential([
    Dense(1024, input_shape=(25388,)),
    Activation('relu'),
    Dense(256),
    Activation('relu'),
    Dense(14),
    Activation('sigmoid'),
])
opt = optimizers.rmsprop(lr=0.0001, decay=1e-6)

#sgd = optimizers.SGD(lr=0.05, decay=1e-6, momentum=0.4, nesterov=False)
model_visual.compile(optimizer=opt,
              loss='binary_crossentropy',
              metrics=['accuracy'])

Using TensorFlow backend.


In [0]:
model_visual.fit(X_train, Y_train, epochs=100, batch_size=64,verbose=0)


In [0]:
Y_preds=model_visual.predict(X_test)


In [0]:
import pickle
f6=open('/content/drive/My Drive/Genredict.pckl','rb')
Genre_ID_to_name=pickle.load(f6)
f6.close()

In [0]:
genre_list=sorted(list(Genre_ID_to_name.keys()))


In [0]:
del f6 
del pickle

In [0]:
sum(sum(Y_preds))


59796.61767578125

In [0]:
import pandas as pd
df = pd.read_pickle("/content/drive/My Drive/pinky_promise_last.pckl")

In [0]:
positions=range(len(df))
# print positions
test_movies=np.asarray(positions)[~mask]
# test_movies

In [0]:
df[df.title=="127 Hours"]

,title,overview,genre_ids,text_vec,poster_path,image_vec
33864,127 Hours,The true story of mountain climber Aron Ralsto...,"[12, 18, 53]","[0.059424083679914474, 0.043944597244262695, -...",/content/drive/My Drive/storage3/127_Hours.jpg,[[[[0.00000000e+00 0.00000000e+00 0.00000000e+...


In [0]:
import numpy as np

precs=[]
recs=[]
for i in range(len(Y_preds)):
    row=Y_preds[i]
    gt_genres=Y_test[i]
    gt_genre_names=[]
    for j in range(14):
        if gt_genres[j]==1:
            gt_genre_names.append(Genre_ID_to_name[genre_list[j]])
    top_3=np.argsort(row)[-4:]
    predicted_genres=[]
    for genre in top_3:
        predicted_genres.append(Genre_ID_to_name[genre_list[genre]])
    (precision,recall)=precision_recall(gt_genre_names,predicted_genres)
    precs.append(precision)
    recs.append(recall)
    print("Predicted: ",','.join(predicted_genres)," Actual: ",','.join(gt_genre_names))

Streaming output truncated to the last 5000 lines.
Predicted:  Fantasy,Thriller,Science Fiction,Mystery  Actual:  Horror,Thriller,Science Fiction,Mystery
Predicted:  Fantasy,Thriller,Science Fiction,Mystery  Actual:  Adventure,Fantasy,Animation,Drama,Action,History,Thriller,Science Fiction,Mystery
Predicted:  Fantasy,Thriller,Science Fiction,Mystery  Actual:  Adventure,Fantasy,Animation,Drama,Horror,Action,History,Western,Thriller,Crime,Science Fiction,Mystery
Predicted:  Adventure,Thriller,Science Fiction,Mystery  Actual:  Adventure,Fantasy,Drama,Action,Western,Crime,Science Fiction,Mystery
Predicted:  Fantasy,Thriller,Science Fiction,Mystery  Actual:  Adventure,Fantasy,Animation,Drama,Comedy,History,Thriller,Crime,Documentary,Science Fiction,Mystery
Predicted:  Fantasy,Thriller,Science Fiction,Mystery  Actual:  Adventure,Fantasy,Animation,Drama,Action,Comedy,History,Thriller,Crime,Science Fiction,Mystery
Predicted:  Fantasy,Thriller,Science Fiction,Mystery  Actual:  Adventure,Fantasy

In [0]:
print(np.mean(np.asarray(precs)),np.mean(np.asarray(recs)))
print("avg=",(np.mean(np.asarray(precs))+np.mean(np.asarray(recs)))/2)

0.8404916889457166 0.4946098969288555
avg= 0.667550792937286
